# Linear Regression & GLMs — Student Lab

Complete all TODOs. Avoid sklearn for core parts.

In [19]:
import numpy as np

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Synthetic Dataset (with collinearity)
We generate data where features can be highly correlated to motivate ridge.

In [6]:
def make_regression(n=400, d=5, noise=0.5, collinear=True):
    X = rng.standard_normal((n, d))
    if collinear and d >= 2:
        X[:, 1] = X[:, 0] * 0.95 + 0.05 * rng.standard_normal(n)
    w_true = rng.standard_normal(d)
    y = X @ w_true + noise * rng.standard_normal(n)
    return X, y, w_true

X, y, w_true = make_regression()
n, d = X.shape
check('shapes', y.shape == (n,))
print('corr(x0,x1)=', np.corrcoef(X[:,0], X[:,1])[0,1])

OK: shapes
corr(x0,x1)= 0.9985704465455699


## Section 1 — OLS Closed Form

### Task 1.1: Closed-form w_hat using solve

# TODO: compute w_hat using solve on (X^T X) w = X^T y
# HINT: `XtX = X.T@X`, `Xty = X.T@y`, `np.linalg.solve(XtX, Xty)`

**Checkpoint:** Why is explicit inverse discouraged?

In [7]:
# Explicit inversion is avoided because it is less stable, slower, and more error-prone than solving the linear system directly.
# Less Stable: Computing inverse magnifies floating point rounding errors, Slower: Matrix inversion computes entire matrix inverse, np.linalg.solve() avoids computing full inverse and only computes the solution which is faster and more efficient.

In [8]:
XtX = X.T @ X                           # Compute feature covariance matrix used in normal equation
Xty = X.T @ y                           # Compute feature–target correlation vector
w_hat = np.linalg.solve(XtX, Xty)       # Solve normal equation to obtain OLS weight estimates

check('w_shape', w_hat.shape == (d,))   # Verify learned weight vector has correct dimension

OK: w_shape


### Task 1.2: Evaluate fit + residuals
Compute:
- predictions y_pred
- MSE
- residual mean and std

**Interview Angle:** What does a structured residual pattern imply (e.g., nonlinearity)?

In [9]:
# A structured residual pattern means the model is biased because it cannot represent the true data-generating process. Linear regression assumes linearity, independent errors, constant variance, zero mean residuals.
# If residuals are not random, the model is determined to be wrong. It's either too simple, missing features or violating assumptions.

In [10]:
y_pred = X @ w_hat                          # Compute model predictions using learned OLS weights
mse = float(np.mean((y_pred - y) ** 2))     # Calculate mean squared error between predictions and targets
resid = y_pred - y                          # Compute residuals (prediction errors)
print('mse', mse, 'resid_mean', resid.mean(), 'resid_std', resid.std()) 
check('finite', np.isfinite(mse))           # Ensure computed loss is a valid finite number

mse 0.2289243168902079 resid_mean 0.029301285107637246 resid_std 0.47756230125633753
OK: finite


## Section 2 — Gradient Descent

### Task 2.1: Implement MSE loss + gradient

Loss = mean((Xw-y)^2), grad = (2/n) X^T(Xw-y)

# TODO: implement `mse_loss_and_grad`

**FAANG gotcha:** shapes and constants.

In [11]:
def mse_loss_and_grad(X, y, w):             # Compute MSE loss and its gradient for linear regression
    r = X @ w - y                           # Compute MSE loss and its gradient for linear regression
    loss = float(np.mean(r * r))            # Compute mean squared error loss
    grad = (2.0 / X.shape[0]) * (X.T @ r)   # Compute gradient via chain rule
    return loss, grad

w0 = np.zeros(d)
loss0, g0 = mse_loss_and_grad(X, y, w0)     # Evaluate loss and gradient at initial weights
check('grad_shape', g0.shape == (d,))
check('finite_loss', np.isfinite(loss0))    # Ensure loss value is finite and valid

OK: grad_shape
OK: finite_loss


### Task 2.2: Train with GD + compare to closed-form

# TODO: implement a simple GD loop, track loss, and compare final weights to w_hat.

**Checkpoint:** How does feature scaling affect GD?

In [12]:
# Feature scaling dramatically improves Gradient Descent because it reshapes the loss surface into a well-conditioned problem, while closed-form solutions don’t depend on convergence speed but still benefit from better numerical stability.

In [13]:
def train_gd(X, y, lr=0.05, steps=500):             # train linear model using gradient descent
    w = np.zeros(X.shape[1])                        # initialize weight vector to zeros
    losses = []                                     # store loss values across iterations
    for _ in range(steps):                          # iterate gradient descent updates
        loss, g = mse_loss_and_grad(X, y, w)        # compute current loss and gradient
        losses.append(loss)                         # track loss to monitor convergence
        w = w - lr * g                              # update weights in negative gradient direction
    return w, losses                                # return weights and loss history

w_gd, losses = train_gd(X, y, lr=0.05, steps=500)   # gradient descent to learn regression weights
print('final_loss', losses[-1])
print('||w_gd-w_hat||', np.linalg.norm(w_gd - w_hat))   # Compare gradient descent solution with closed-form OLS solution
check('loss_decreases', losses[-1] <= losses[0])

final_loss 0.23136626716013964
||w_gd-w_hat|| 1.376975427170424
OK: loss_decreases


## Section 3 — Ridge Regression (L2)

### Task 3.1: Ridge closed-form
w = (X^T X + λI)^{-1} X^T y

# TODO: implement ridge_solve

**Interview Angle:** Why does ridge help under collinearity?

In [14]:
# Ridge helps under collinearity because adding 𝜆𝐼 (lam) increases small eigenvalues (Ridge pushes all eigenvalues away from zero), stabilizes matrix inversion (Matrix is no longer near-singular), reduces variance (Adds small bias), and prevents coefficient explosion.

In [15]:
def ridge_solve(X, y, lam):
    d = X.shape[1]                  # number of features
    return np.linalg.solve(         
        X.T @ X + lam * np.eye(d),  # adds L2 regularization for stability
        X.T @ y)                    # correlation between features and targets


w_ridge = ridge_solve(X, y, lam=1.0)        # compute ridge-regularized weight vector
check('ridge_shape', w_ridge.shape == (d,)) # ensure output has one weight per feature

OK: ridge_shape


### Task 3.2: Bias/variance demo with train/test split

# TODO: split into train/test and compare MSE for multiple lambdas.

**Checkpoint:** why can test error improve even when train error worsens?

In [16]:
# Test error improves while train error worsens because regularization reduces variance more than it increases bias, improving generalization. Training error always prefers complexity. Test error prefers the right balance. You trade a little bias for a big reduction in variance when lam is added.

In [17]:
# TODO
idx = rng.permutation(n)
train = idx[: int(0.7*n)]
test = idx[int(0.7*n):]
Xtr, ytr = X[train], y[train]
Xte, yte = X[test], y[test]

lams = [0.0, 0.1, 1.0, 10.0]
results = []
for lam in lams:
    w = ridge_solve(Xtr, ytr, lam=lam) if lam > 0 else np.linalg.solve(Xtr.T@Xtr, Xtr.T@ytr)
    tr_mse = np.mean((Xtr@w - ytr)**2)
    te_mse = np.mean((Xte@w - yte)**2)
    results.append((lam, tr_mse, te_mse))
print('lam, train_mse, test_mse')
for r in results:
    print(r)

lam, train_mse, test_mse
(0.0, np.float64(0.21799841512243526), np.float64(0.257745393681019))
(0.1, np.float64(0.21809342046539712), np.float64(0.2584161606858411))
(1.0, np.float64(0.219110917219492), np.float64(0.2610634849213479))
(10.0, np.float64(0.22348087752854684), np.float64(0.2703372913154806))


## Section 4 — GLM Intuition

### Task 4.1: Match tasks to (distribution, link)
Fill in a table for:
- regression
- binary classification
- count prediction

**Explain:** what changes when you go from OLS to a GLM?

| Problem | Target type | Distribution | Link | Loss |
|---|---|---|---|---|
| House price | continuous | Gaussian (Normal) | Identity | MSE (Squared error) |
| Fraud | binary | Bernoulli | Sigmoid | Log Loss (cross-entropy) |
| Clicks per user | count | Poisson | Log | Poisson Deviance |


In [18]:
# GLM lets you choose distribution based on target type:
# Gaussian → continuous
# Bernoulli → binary
# Poisson → counts
# So the noise model changes.
# GLMs generalize OLS by allowing different distributions and link functions, making linear modeling work for binary and count data, not just continuous targets. GLM extends linear modeling to Probabilities (Fraud detection), counts (clicks, events), rates and many real world problems while OLS works for continuous outcomes.

---
## Submission Checklist
- All TODOs completed
- Train/test results shown for ridge
- Short answers to checkpoint questions
